### - 데이터 수집

#### # 최근 6개월 후기(제목, 시간, 별점, 날짜, 후기)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime, timedelta
# Chrome WebDriver 경로 설정 (본인 환경에 맞게 수정 필요)
driver = webdriver.Chrome()
# 웹 페이지 로드
driver.get('https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2')
# 데이터를 저장할 리스트 초기화
titles = []
times = []
ratings = []
dates = []
contents = []
# 오늘 날짜 기준으로 6개월 전 날짜 계산
today = datetime.now()
six_months_ago = today - timedelta(days=180)
page = 1
continue_crawling = True
while page <= 315 and continue_crawling:
    try:
        # 페이지 버튼의 XPath 계산
        if page % 10 == 1:
            page_xpath = '//*[@id="mainForm"]/div[3]/strong/a'
        else:
            page_in_group = (page - 1) % 10 + 2
            page_xpath = f'//*[@id="mainForm"]/div[3]/a[{page_in_group}]'
        # 페이지 버튼 클릭하여 해당 페이지로 이동
        page_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, page_xpath))
        )
        page_button.click()
        # 현재 페이지 번호 출력
        print(f"현재 페이지: {page}")
        # 각 페이지의 10개 요소 순회
        for row in range(1, 11):
            try:
                # 날짜 추출
                date_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[3]'
                date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, date_xpath))
                )
                date_text = date_element.text.strip().split('\n')[-1]
                review_date = datetime.strptime(date_text, '%Y.%m.%d')
                # 6개월 이내의 후기만 크롤링
                if review_date >= six_months_ago:
                    dates.append(date_text)
                    # 제목 추출
                    title_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a'
                    title_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, title_xpath))
                    )
                    title = title_element.text.split('<br>')[0].strip()
                    title = title.split('시간 ')[1].split('\n\n')[0]
                    titles.append(title)
                    # 본문 추출
                    content_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a'
                    content_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, content_xpath))
                    )
                    content_html = content_element.get_attribute('innerHTML')
                    content = content_html.split('<br>\n')[2].strip().split('<img')[0].strip()
                    contents.append(content)
                    # 시간 추출
                    time_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a/span'
                    time_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, time_xpath))
                    )
                    time = time_element.text.strip().replace('시간','')
                    times.append(time)
                    # 별점(평점) 추출
                    rating_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[3]/img'
                    rating_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, rating_xpath))
                    )
                    rating_alt = rating_element.get_attribute('alt')
                    ratings.append(rating_alt)
                else:
                    print(f"6개월 이내의 후기가 아닙니다: {date_text}")
                    continue_crawling = False
                    break
            except Exception as e:
                print(f"페이지 데이터 추출 중 오류 발생 (행 {row}):", e)
                continue
        # 다음 페이지 번호로 이동
        if page % 10 == 0 and page <= 310 and continue_crawling:  # 페이지 그룹의 마지막 페이지이고 310 페이지 이하인 경우
            next_page_button_xpath = '//*[@id="mainForm"]/div[3]/a[12]'
            next_page_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, next_page_button_xpath))
            )
            next_page_button.click()
        page += 1
    except Exception as e:
        print("전체 프로세스 중 오류 발생:", e)
        break
# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame({
    '제목': titles,
    '시간': times,
    '별점': ratings,
    '날짜': dates,
    '본문': contents
})
# 결과 출력
print(result_df)
# 크롬 드라이버 종료
driver.quit()

현재 페이지: 1
현재 페이지: 2
현재 페이지: 3
현재 페이지: 4
현재 페이지: 5
현재 페이지: 6
현재 페이지: 7
현재 페이지: 8
현재 페이지: 9
현재 페이지: 10
현재 페이지: 11
현재 페이지: 12
현재 페이지: 13
현재 페이지: 14
현재 페이지: 15
6개월 이내의 후기가 아닙니다: 2023.12.17
                                제목  시간  별점          날짜  \
0                   수업을 살리는 체육 레시피  30  5점  2024.06.13   
1                교사를 위한 교실 밖 여행인문학  15  5점  2024.06.10   
2                    왕초보를 위한 캘리그래피  15  5점  2024.06.10   
3          그림검사와 다양한 기법으로 만나는 미술치료  30  5점  2024.06.07   
4              같이 읽고 함께 나누는 독서수업방법  30  5점  2024.06.06   
..                             ...  ..  ..         ...   
142       손그림과 파워포인트로 교실 속 콘텐츠 만들기  30  5점  2023.12.19   
143  학급경영, 수업, 업무 - 노션으로 한 방에 해결하기  15  5점  2023.12.19   
144         아이의 마음을 읽는 다양한 미술치료 기법  15  5점  2023.12.18   
145       English Grammar in Use 2  30  4점  2023.12.18   
146  학급경영, 수업, 업무 - 노션으로 한 방에 해결하기  15  5점  2023.12.18   

                                     본문  
0    아이들이 정~~~말 좋아하는 체육을 더 더 즐겁게 해주기 위…  
1    저는 여행에 관심이 많은 교사입니다. 학기중 바쁜 일

In [2]:
result_df

,제목,시간,별점,날짜,본문
0,수업을 살리는 체육 레시피,30,5점,2024.06.13,아이들이 정~~~말 좋아하는 체육을 더 더 즐겁게 해주기 위…
1,교사를 위한 교실 밖 여행인문학,15,5점,2024.06.10,저는 여행에 관심이 많은 교사입니다. 학기중 바쁜 일정으로 …
2,왕초보를 위한 캘리그래피,15,5점,2024.06.10,"미술시간에 아이들과 캘리그라피 해봤었는데,이렇게 자세히 알려…"
3,그림검사와 다양한 기법으로 만나는 미술치료,30,5점,2024.06.07,학생들이 감정이나 내면세계를 미술 활동을 통해 표현하므로 학…
4,같이 읽고 함께 나누는 독서수업방법,30,5점,2024.06.06,워낙 독서토론 분야에서 유명하신 김성현 선생님 강의가 티셀파…
...,...,...,...,...,...
142,손그림과 파워포인트로 교실 속 콘텐츠 만들기,30,5점,2023.12.19,어러웠지만 재미있는 연수였습니다. 겨울 방학 동안에 만화 작…
143,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,5점,2023.12.19,현장에서 실제 적용해보기 위한 여러 시도를 해보게 되었던 시…
144,아이의 마음을 읽는 다양한 미술치료 기법,15,5점,2023.12.18,실제적인 강의가 많은 도움이 되었습니다. \n학교뿐 아니라 …
145,English Grammar in Use 2,30,4점,2023.12.18,저는 혼자서 문법을 정리하고 싶어 올해 초 교재를 사서 혼자…


#### # 직무연수 목록(제목, 연수 분야, 가격, 시간, 대상)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import pandas as pd
# 웹 드라이버 초기화
driver = webdriver.Chrome()
try:
    # 페이지 접속
    driver.get('https://edu.tsherpa.co.kr/Product/List/')
    # 데이터 저장을 위한 리스트 초기화
    titles = []
    times = []
    peoples = []
    pays = []
    fields = []
    # 각 요소에 대한 데이터를 추출
    for i in range(1, 82):  # 1부터 81까지
        try:
            field = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[1]'))
            ).text
            fields.append(field)
        except (TimeoutException, NoSuchElementException):
            fields.append(float('nan'))
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/p[2]/a'))
            ).text
            titles.append(title)
        except (TimeoutException, NoSuchElementException):
            titles.append(float('nan'))
        try:
            time = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/p[1]/span[1]'))
            ).text
            times.append(time)
        except (TimeoutException, NoSuchElementException):
            times.append(float('nan'))
        try:
            price_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[4]/p[1]'))
            )
            # Extract text and format price if it contains "100% 할인"
            price_text = price_elem.text.strip()
            if "100%할인\n0 원" in price_text:
                price = "0원"  # Set to "0원" if 100% discount
            else:
                price = price_text  # Use as-is if no special formatting
            pays.append(price)
        except (TimeoutException, NoSuchElementException):
            pays.append(float('nan'))
        try:
            target = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/div/p[2]'))
            ).text
            peoples.append(target)
        except (TimeoutException, NoSuchElementException):
            peoples.append(float('nan'))
    # 데이터 프레임 생성
    df = pd.DataFrame({
        '연수 분야': fields,
        '제목': titles,
        '시간': times,
        '가격': pays,
        '대상': peoples
    })
    # 결과 출력
    print(df)
except WebDriverException as e:
    print("웹드라이버 오류:", e)
finally:
    driver.quit()

     연수 분야                             제목    시간        가격              대상
0     교과수업                 교사가 알고싶은 독립운동사  20시간        0원              전체
1    교과서연수    실패 없는 과학 교과서 실험 (3~4학년 1학기)  10시간        0원              초등
2   ICT정보화    쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!  30시간  70,000 원              전체
3     인문교양           학생들과 함께 나누는 기후변화 이야기  30시간  70,000 원              전체
4   ICT정보화  학급경영, 수업, 업무 - 노션으로 한 방에 해결하기  15시간  40,000 원  초등 중등 교육전문직 일반
..     ...                            ...   ...       ...             ...
76    교과수업                 수업을 살리는 미술 레시피  30시간  70,000 원        유치 초등 일반
77    교과수업                 수업을 살리는 꿀팁 레시피  30시간  70,000 원        초등 교육전문직
78    자기계발                    나만의 여행책 만들기  30시간  70,000 원              전체
79    교과수업                 수업을 살리는 놀이 레시피  30시간  70,000 원     초등 교육전문직 일반
80    교과수업                교과수업에 활용하는 진로교육  30시간  70,000 원     초등 중등 교육전문직

[81 rows x 5 columns]


In [4]:
df

,연수 분야,제목,시간,가격,대상
0,교과수업,교사가 알고싶은 독립운동사,20시간,0원,전체
1,교과서연수,실패 없는 과학 교과서 실험 (3~4학년 1학기),10시간,0원,초등
2,ICT정보화,쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!,30시간,"70,000 원",전체
3,인문교양,학생들과 함께 나누는 기후변화 이야기,30시간,"70,000 원",전체
4,ICT정보화,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15시간,"40,000 원",초등 중등 교육전문직 일반
...,...,...,...,...,...
76,교과수업,수업을 살리는 미술 레시피,30시간,"70,000 원",유치 초등 일반
77,교과수업,수업을 살리는 꿀팁 레시피,30시간,"70,000 원",초등 교육전문직
78,자기계발,나만의 여행책 만들기,30시간,"70,000 원",전체
79,교과수업,수업을 살리는 놀이 레시피,30시간,"70,000 원",초등 교육전문직 일반


#### # 최근 6개월동안의 후기 최종 데이터(제목, 연수 분야, 시간, 후기, 별점, 날짜, 가격, 대상)

In [18]:
fin_res = pd.merge(result_df, df, left_on='제목', right_on='제목')

In [19]:
fin_res = fin_res.drop(columns=['시간_y']).rename(columns={'시간_x':'시간'})

In [20]:
fin_res

,제목,시간,별점,날짜,본문,연수 분야,가격,대상
0,수업을 살리는 체육 레시피,30,5점,2024.06.13,아이들이 정~~~말 좋아하는 체육을 더 더 즐겁게 해주기 위…,교과수업,"70,000 원",유치 초등 교육전문직 일반
1,수업을 살리는 체육 레시피,30,5점,2024.04.09,간혹 학교에 물품이 없어서 하지 못하는 체육 활동들에 대한 …,교과수업,"70,000 원",유치 초등 교육전문직 일반
2,수업을 살리는 체육 레시피,30,5점,2024.01.22,새학기 재미있는 체육 수업을 위해 ~,교과수업,"70,000 원",유치 초등 교육전문직 일반
3,수업을 살리는 체육 레시피,30,5점,2024.01.16,수업하다가 학생들이 가끔 싫어하는 단원들도 있어 이로 응용해…,교과수업,"70,000 원",유치 초등 교육전문직 일반
4,교사를 위한 교실 밖 여행인문학,15,5점,2024.06.10,저는 여행에 관심이 많은 교사입니다. 학기중 바쁜 일정으로 …,인문교양,"40,000 원",전체
...,...,...,...,...,...,...,...,...
136,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,5점,2023.12.18,노션의 활용방법에서 기본적인 세팅 외에 업무적으로 현장에서 …,ICT정보화,"40,000 원",초등 중등 교육전문직 일반
137,나만의 여행책 만들기,30,5점,2024.01.19,세계 여행을 다녀온 것 같은 기분이 드는 연수입니다.\n사진…,자기계발,"70,000 원",전체
138,어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년),10,5점,2024.01.02,사회수을 처음부터 어떻게 계획하고 이끌어 가야하는지 잘 설명…,교과서연수,0원,초등 교육전문직 일반
139,어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년),10,5점,2023.12.22,짧은 시간의 연수지만 알차고 도움이 많이 되었습니다.감사합니…,교과서연수,0원,초등 교육전문직 일반


In [21]:
# 최근 6개월 후기 csv 파일로 저장
fin_res.to_csv('./T_review.csv', index=False, encoding='utf-8-sig')

---

### - 기초 분석

#### # 연수 과정별 수강후기 개수

In [22]:
title_counts = fin_res['제목'].value_counts().reset_index()
title_counts.columns = ['제목', '수강후기 개수']

In [23]:
title_counts

,제목,수강후기 개수
0,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",8
1,쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!,8
2,교사가 알고싶은 독립운동사,6
3,손그림과 파워포인트로 교실 속 콘텐츠 만들기,6
4,지리와 함께하는 인문학 티타임,6
5,세상에서 제일 쉬운 과학 이야기,6
6,"기후 변화 이야기, 교실에 오다",5
7,쉽게 배우고 활용하는 교실 속 손그림 그리기,5
8,메이슨의 Travel English,5
9,수업을 살리는 체육 레시피,4


#### # 연수 분야별 수강후기 개수

In [24]:
field_counts = fin_res['연수 분야'].value_counts().reset_index()
field_counts.columns = ['연수 분야', '후기 개수']

field_counts

,연수 분야,후기 개수
0,자기계발,43
1,ICT정보화,29
2,교과수업,26
3,어학,20
4,인문교양,11
5,생활지도,6
6,교과서연수,4
7,학습지도,1
8,학급경영,1
